In [ ]:
import os
from PIL import Image
import torch

from diffusers import QwenImageEditPlusPipeline

model_path = "ovedrive/Qwen-Image-Edit-2509-4bit"
pipeline = QwenImageEditPlusPipeline.from_pretrained(model_path, torch_dtype=torch.bfloat16)
print("pipeline loaded") # not true but whatever. do not move to cuda

pipeline.set_progress_bar_config(disable=None)
pipeline.enable_model_cpu_offload()
image1 = Image.open("data/florian_0050.jpg").convert("RGB")
image2 = Image.open("test_bild_shirt.jpg").convert("RGB")
prompt = "Make this person wear the shirt instead of his original clothing. Do not invent new features other then the clothing change. Only replace the original clothing item with the new shirt, while keeping the person exactly the same. keep the pose, body, face, hair, and surroundings exactly the same."
inputs = {
    "image": [image1, image2],
    "prompt": prompt,
    "generator": torch.manual_seed(0),
    "true_cfg_scale": 4.0,
    "negative_prompt": "change pose",
    "num_inference_steps": 20, # even 10 steps should be enough in many cases
}

with torch.inference_mode():
  output = pipeline(**inputs)

output_image = output.images[0]
output_image.save("output_image_flo2.png")
print("image saved at", os.path.abspath("flo_output/output_image_flo2.png"))
